# Instructions:
1) In 'Specify Input/ Output Directories' code block, update: INPUT_PATH_FOLDER and OUTPUT_PATH_FOLDER
2) Run all cells in notebook

If you are having issue (or are working on this code) see the developer notes section at the end

## Import Libraries/ Functions:

In [1]:
#general library:
import os
import shutil

#add sextractor to path
import sys
sys.path.insert(0, '../gibbi')

#now we can import the star_removal method:
from star_removal import make_star_mask_from_sextractor_file
from fits import write_fits, read_fits

## Specify Input/ Output Directories:

In [2]:
################################  IMPORTANT ################################:
#modify these paths:

#path to folder (as it appears on your native OS)

INPUT_PATH_FOLDER = '../../spin-parity-catalog/output_from_running/sextractor/table2'
OUTPUT_PATH_FOLDER = '../../spin-parity-catalog/image_processing/star_mask/table2'
FITS_PATH_FOLDER =  '../../spin-parity-catalog/galaxies/table2'

#########################################################################

In [3]:
#check input directory:
if not os.path.isdir(INPUT_PATH_FOLDER):
    print('INPUT_PATH_FOLDER = {}: does not exist, double check the INPUT_PATH_FOLDER variable (HINT: you have to modify this)'.format(INPUT_PATH_FOLDER))
   
#check if output directory exists, and if not make it:
if os.path.isdir(OUTPUT_PATH_FOLDER):
    pass
elif OUTPUT_PATH_FOLDER != '':
    os.makedirs(OUTPUT_PATH_FOLDER)
else:
    print('OUTPUT_PATH_FOLDER = {}: is blank, double check the OUTPUT_PATH_FOLDER variable (HINT: you have to modify this)'.format(OUTPUT_PATH_FOLDER))
    
#checks fits path:
if not os.path.isdir(FITS_PATH_FOLDER):
    print('FITS_PATH_FOLDER = {}: does not exist, double check the FITS_PATH_FOLDER variable (HINT: you have to modify this)'.format(FITS_PATH_FOLDER))
 

## run sextractor on all fits files in nested INPUT_PATH_FOLDER:

In [4]:
def get_sex_files_in_folder(folder_path):
    fits_files = []
    for file in os.listdir(folder_path):
        if file.endswith(".sex"):
            fits_files.append(file)
    return fits_files

In [5]:
def run_sextractor_with_all_fits(input_folder,output_folder,fits_folder):
    for gal_name in os.listdir(input_folder):
        
        #Step 1: Create Output dir for specific galaxy
        print('Running on {}'.format(gal_name))
        gal_output_path = os.path.join(output_folder,gal_name)
        os.makedirs(gal_output_path,exist_ok=True) #requires python 3.2+
        
        #Step 2: Get all sextractor files for galaxy
        sex_files =  get_sex_files_in_folder(os.path.join(input_folder,gal_name))
        
        #Step 3: Iterate through fits and run_sextractor on all fits
        for sex_file in sex_files:
            #Step 3a: Get input_sextractor_path, output_star_mask_path, and fits_path
            # i.e:
            #     for sextractor file 'IC1683/IC1683_g.sex' in directory <input_folder>:
            #     input_sextractor_path = '<input_folder>/IC1683/IC1683_g.sex'
            #     output_star_mask_path = '<output_folder>/IC1683/IC1683_g_star_mask.fit
            #     fits_path = '<fits_folder>/IC1683/IC1683_g.fits'
            
            star_mask_name = "{}_star_mask.fits".format(sex_file.split('.')[0])
            fits_name = "{}.fits".format(sex_file.split('.')[0])
            
            input_sextractor_path = os.path.join(input_folder,gal_name,sex_file)
            output_star_mask_path = os.path.join(output_folder,gal_name,star_mask_name)
            fits_path = os.path.join(fits_folder,gal_name,fits_name)
            
            #Step 3b: Get shape of fits
            fits_shape = read_fits(fits_path).shape
            
            #Step 3c: Create star mask and save it to output_star_mask_path
            star_mask = make_star_mask_from_sextractor_file(input_sextractor_path,fits_shape,min_star_class=0.1)
            write_fits(output_star_mask_path,star_mask)

In [6]:
run_sextractor_with_all_fits(INPUT_PATH_FOLDER,OUTPUT_PATH_FOLDER,FITS_PATH_FOLDER)

Running on NGC3627
Running on NGC6503
Running on NGC4451
Running on MCG-02-51-004
Running on NGC5980
Running on NGC3672
Running on NGC6132
Running on NGC4450
Running on NGC3675
Running on NGC3227
Running on NGC6907
Running on NGC7479
Running on NGC4698
Running on NGC1637
Running on NGC3160
Running on NGC2280
Running on UGC1057
Running on NGC5033
Running on UGC10331
Running on NGC7331
Running on NGC5005
Running on NGC1035
Running on NGC1056
Running on NGC949
Running on NGC7364
Running on NGC1093
Running on NGC7738
Running on NGC4310
Running on UGC7901
Running on IC1755
Running on MCG-02-02-030
Running on NGC4321
Running on NGC6015
Running on NGC3900
Running on UGC9892
Running on NGC7782
Running on UGC36
Running on NGC1421
Running on UGC10384
Running on NGC4501
Running on UGC7145
Running on NGC157
Running on NGC3175
Running on NGC3949
Running on UGC9873
Running on NGC5678
Running on NGC3312
Running on NGC5248
Running on NGC169
Running on NGC5426
Running on NGC3521
Running on IC5376
Runni

In [8]:
#Check size of input and output (and figure out if there was an issue running on something)
def compare_input_and_output(input_folder,output_path):
    input_count = 0
    output_count = 0
    for gal_name in os.listdir(input_folder):
        for file in os.listdir(os.path.join(input_folder,gal_name)):
            if file.endswith(".sex"):
                input_count += 1
                
                star_mask_name = "{}_star_mask.fits".format(file.split('.')[0])
                output = os.path.join(output_path,gal_name,star_mask_name)
                if os.path.exists(output):
                    output_count += 1
                else:
                    print('Missing {}'.format(star_mask_name))
    print("Input: {} sextractor files, Output: {} star masks".format(input_count,output_count))
    
compare_input_and_output(INPUT_PATH_FOLDER,OUTPUT_PATH_FOLDER)

Input: 675 sextractor files, Output: 675 star masks


## Developer Notes:
Last Updated: 06/29/2022
 
Known Bugs:
1) If source extractor objects has a center outside of bounds of fits image, function `create_elliptical_mask` in `mask.py` returns nothing
For example: `NGC3312` has an object with dict = `{'XWIN_IMAGE': 765.473, 'YWIN_IMAGE': -7.86, ...}`
Workaround: In function filter_objects in star_removal.py, added Step 5 to filter out centers outside of bounds

Things to Look out for:
1) The biggest potential issue is photoutil and source extractor use different coordinate systems.
I added this section in `mask.py` just to make it super clear:

##########################################################################################################################
NOTE:                                                                                                                  #       
Elliptical apperature uses the following coordinate convention:                                                      #
In Photutils, pixel coordinates are zero-indexed, meaning that (x, y) = (0, 0) corresponds to the center of the   #
lowest, leftmost array element. This means that the value of data[0, 0] is taken as the value over the range      # 
-0.5 < x <= 0.5, -0.5 < y <= 0.5.                                                                                 #
Source: https://buildmedia.readthedocs.org/media/pdf/photutils/v0.3/photutils.pdf                                   #
Documentation:https://photutils.readthedocs.io/en/stable/api/photutils.aperture.EllipticalAperture.html             #

This differs from the SourceExtractor, IRAF, FITS, and ds9 conventions, in                                           #
which the center of the lowest, leftmost array element is (1, 1).                                                 #
Source: See pg.59 in http://star-www.dur.ac.uk/~pdraper/extractor/Guide2source_extractor.pdf                       #
##########################################################################################################################

This is why when you see me `'YWIN_IMAGE'` or `'XWIN_IMAGE'` I have `object_dict['YWIN_IMAGE'] - 1` and `object_dict['XWIN_IMAGE'] - 1`.

2) Source Extractor outputs theta in degrees but photoutils uses radians. Hence in `star_removal.py` when papssing in theta to `create_elliptical_mask` I have `np.radians(gal_dict['THETA_IMAGE'])`

3) I am binarizing the star mask (any value more than 0 -> 1). This is also why the line `the_final_mask[the_final_mask>0] = 1.0` is there